In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Creating dummy test set (you may skip this section)

In [15]:
file_name = "Deloitte_DrugDiscovery_dataset.csv"
# Open file
df = pd.read_csv(file_name)

# Create a fake test set, select 100 random rows
test_set = df.sample(100, random_state=42)
# Save the test set to a csv file
test_set.to_csv("Data/dummy_test_set.csv", index=False)

# Open test set
Change file name to the test set you would like to use

In [16]:
# Open the test set
test_set = pd.read_csv("Data/dummy_test_set.csv")
test_set.head()

,UniProt_ID,pubchem_cid,kiba_score,kiba_score_estimated
0,P00811,68206073.0,715.0,True
1,P35354,118705851.0,26.3,True
2,Q8WTS6,122192761.0,14600.0,True
3,O75116,57446057.0,126.0,True
4,P41597,71214976.0,17.8,True


# Get protein embeddings

In [1]:
import os

# Specify the folder where you want to save the file
folder = "Embeddings"

# Create the folder if it doesn't exist
os.makedirs(folder, exist_ok=True)

# Define the file URL and the destination path
url = "https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/embeddings/uniprot_sprot/per-protein.h5"
destination = os.path.join(folder, "per-protein.h5")

# Download the file to the specified folder
os.system(f"wget -O {destination} {url}")

print(f"File downloaded and saved to {destination}")

File downloaded and saved to Embeddings\per-protein.h5


In [14]:
import h5py

protein_sequences = list(test_set["UniProt_ID"].values)
# protein_sequences = ['abc', 'def', 'ghi', 'jkl', 'mno']

embeddings = []
with h5py.File("Embeddings/per-protein.h5", "r") as file:
    for sequence_id in protein_sequences:
        try:
            embedding = file[sequence_id][:]
            embeddings.append(embedding)
        except KeyError:
            # If the sequence_id is not found, append a NaN of 1024 dimensions
            embeddings.append([np.nan] * 1024)
            print(f"Warning: Sequence ID {sequence_id} not found in the file.")
            continue  # Skip to the next sequence_id if not found

embeddings = np.array(embeddings)
print(f"Embeddings shape: {embeddings.shape}")
# Turn into dataframe
df_prot_embeddings = pd.DataFrame(embeddings)
# Index column should be the UniProt_ID
df_prot_embeddings.index = protein_sequences
# Name it "UniProt_ID"
df_prot_embeddings.index.name = "UniProt_ID"
# Column names should be prot_0, prot_1, ..., prot_767
df_prot_embeddings.columns = [f"prot_{i}" for i in range(embeddings.shape[1])]
df_prot_embeddings.head()

Embeddings shape: (100, 1024)


,prot_0,prot_1,prot_2,prot_3,prot_4,prot_5,prot_6,prot_7,prot_8,prot_9,...,prot_1014,prot_1015,prot_1016,prot_1017,prot_1018,prot_1019,prot_1020,prot_1021,prot_1022,prot_1023
UniProt_ID,,,,,,,,,,,,,,,,,,,,,
P00811,0.041260,0.121460,0.046600,0.029480,-0.024811,0.009254,-0.031006,-0.054626,0.088074,-0.017487,...,-0.043335,-0.027161,0.007919,-0.029633,0.079102,-0.037720,-0.071472,-0.044373,-0.046204,0.011169
P35354,-0.005409,0.032898,-0.010170,-0.002161,-0.039429,0.009239,-0.000806,0.028351,0.021088,0.012329,...,-0.100220,0.050934,0.014870,-0.062073,0.056732,0.030273,-0.003635,-0.018616,0.009300,0.068848
Q8WTS6,0.045258,0.038849,0.047760,0.042358,-0.016830,0.003963,0.002913,-0.062744,0.009666,-0.040802,...,-0.003094,-0.028183,-0.001230,-0.043945,0.079895,-0.005993,-0.020737,-0.024689,-0.026596,-0.015221
O75116,0.021347,0.037109,-0.002827,0.005844,0.018707,0.020004,0.003159,-0.046295,0.031433,-0.019501,...,-0.027496,-0.061249,-0.024277,0.000149,0.067627,-0.003065,-0.032410,0.006607,-0.012283,0.003008
P41597,-0.023102,0.090210,0.039612,0.006996,-0.023697,0.006744,0.003416,-0.029129,0.007301,-0.006870,...,-0.009850,-0.003531,0.016724,-0.037140,0.024857,-0.021683,0.012924,-0.003733,0.000465,0.013481


# Get molecule embeddings
If the pubchem_cids from the test set can also be found in the Deloitte_DrugDiscovery_dataset, the SMILES representation can be found in the cache. <br>
If not, the SMILES representation can be pulled from PubChem.

## Option 1: Get SMILES from cache

In [8]:
# Get cids from test_set
test_cids = list(test_set["pubchem_cid"].values.astype(int))

# Load existing SMILES database if available
cache_file = "Data/smiles_cache.csv"
if os.path.exists(cache_file):
    smiles_cache = pd.read_csv(cache_file)
    smiles_cache_dict = dict(zip(smiles_cache["CID"], smiles_cache["SMILES"]))
    print(f"Loaded {len(smiles_cache)} SMILES from cache.")

# Find pubchem_cids that are not in the cache
missing_cids = [cid for cid in test_cids if cid not in smiles_cache_dict]
print(f"Number of missing CIDs: {len(missing_cids)}")
print(f"Missing CIDs: {missing_cids}")

# Create a subdictionary from smiles_cache_dict with only the test_cids
test_smiles = {cid: smiles_cache_dict[cid] for cid in test_cids if cid in smiles_cache_dict}

Loaded 683413 SMILES from cache.
Number of missing CIDs: 0
Missing CIDs: []


## Option 2: Pull SMILES from PubChem

In [10]:
from fetch_smiles import fetch_smiles_parallel

test_smiles, failed_cids = fetch_smiles_parallel(test_cids, batch_size=100, retries=3, timeout=10)
print(f"Number of failed CIDs: {len(failed_cids)}")


Failed to retrieve SMILES for 0 CIDs.
Number of failed CIDs: 0


## Create embedding (Morgan Fingerprint)

In [13]:
from rdkit import Chem
from rdkit.Chem import AllChem

# Example list of SMILES strings
# smiles_list = ["CCO", "CC(=O)O", "CC(C)O"]
smiles_list = list(test_smiles.values())

# Function to generate Morgan fingerprints
def compute_morgan_fingerprint(smiles, radius=2, n_bits=1024):
    try:
        mol = Chem.MolFromSmiles(smiles)  # Convert SMILES to RDKit molecule
        if mol is None:
            raise ValueError(f"Invalid SMILES: {smiles}")
        # Compute fingerprint
        fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
        return np.array(fp)  # Convert to NumPy array
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None

# Compute fingerprints for the SMILES list
fingerprints = [compute_morgan_fingerprint(smiles) for smiles in smiles_list]

fingerprints = np.array(fingerprints)
print(f"Fingerprints shape: {fingerprints.shape}")

# Turn into dataframe
fingerprints_df = pd.DataFrame(fingerprints)
# Add CID column
fingerprints_df['CID'] = list(test_smiles.keys())
# Make CID the index
fingerprints_df.set_index('CID', inplace=True)
# Name the cols mol_0, mol_1, etc
fingerprints_df.columns = [f'mol_{i}' for i in range(fingerprints_df.shape[1])]
fingerprints_df.head()

Fingerprints shape: (100, 1024)


[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerator
[05:46:17] DEPRECATION WARNING: please use MorganGenerat

,mol_0,mol_1,mol_2,mol_3,mol_4,mol_5,mol_6,mol_7,mol_8,mol_9,...,mol_1014,mol_1015,mol_1016,mol_1017,mol_1018,mol_1019,mol_1020,mol_1021,mol_1022,mol_1023
CID,,,,,,,,,,,,,,,,,,,,,
68206073,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
118705851,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
122192761,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
57446057,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
71214976,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


# Prepare for ML Pipeline

In [27]:
# Protein embeddings preprocessing
# Convert to float16
protein_embeddings = df_prot_embeddings.astype(np.float16)
# Normalize the embeddings
protein_embeddings = (protein_embeddings - protein_embeddings.mean()) / protein_embeddings.std()

# Molecule embeddings preprocessing
# Convert to int8
mol_embeddings = fingerprints_df.astype(np.int8)
# Make sure index of molecule embeddings is string
mol_embeddings.index = mol_embeddings.index.astype(int)

print("Merging datasets...")
df_embedded = test_set.merge(protein_embeddings, left_on="UniProt_ID", right_index=True)
df_embedded = df_embedded.merge(mol_embeddings, left_on="pubchem_cid", right_index=True)
print(f"Merged dataset: {df_embedded.shape}")

# Remove rows with NaN values
df_embedded.dropna(inplace=True)
print(f"After removing NaN values: {df_embedded.shape}")

df_embedded.loc[:, 'kiba_score_estimated'] = df_embedded['kiba_score_estimated'].astype(np.int8)

df_embedded.head()

Merging datasets...
Merged dataset: (118, 2052)
After removing NaN values: (117, 2052)


,UniProt_ID,pubchem_cid,kiba_score,kiba_score_estimated,prot_0,prot_1,prot_2,prot_3,prot_4,prot_5,...,mol_1014,mol_1015,mol_1016,mol_1017,mol_1018,mol_1019,mol_1020,mol_1021,mol_1022,mol_1023
0,P00811,68206073.0,715.0,1,0.705078,1.677734,0.836914,0.833496,-0.540527,-0.887695,...,0,0,0,0,1,1,0,0,0,0
1,P35354,118705851.0,26.3,1,-1.000977,-0.895508,-1.636719,-0.621094,-1.139648,-0.889160,...,0,0,0,0,0,1,0,0,0,0
2,Q8WTS6,122192761.0,14600.0,1,0.851074,-0.722656,0.887695,1.425781,-0.213623,-1.103516,...,0,0,0,0,0,0,0,0,0,0
3,O75116,57446057.0,126.0,1,-0.022873,-0.772949,-1.316406,-0.252930,1.243164,-0.449463,...,0,0,0,0,0,1,0,0,0,1
3,O75116,57446057.0,126.0,1,-0.022873,-0.772949,-1.316406,-0.252930,1.243164,-0.449463,...,0,0,0,0,0,1,0,0,0,1


In [32]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score

# Define the target variable and the features
X_test = df_embedded.drop(columns=['UniProt_ID', 'pubchem_cid', 'kiba_score'])  # The embeddings (protein and molecule features)
y_test = df_embedded['kiba_score']  # The target variable

# Convert features and target to float32
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

# Print the size of X_test
print("Size of X:", X_test.shape)    

# Load model 'Models/best_lgbm_model_full.txt'
model = lgb.Booster(model_file='Models/best_lgbm_model_full.txt')

# Predict on the test set using the best model
y_pred = model.predict(X_test, num_iteration=model.best_iteration)

# Evaluate the performance using Mean Squared Error and R^2 Score
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R^2 Score:", r2_score(y_test, y_pred))

Size of X: (117, 2049)
Mean Squared Error: 15390161240.018251
R^2 Score: -8.503723586757072
